* Connecting with Langchain

In [10]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display
from openai import OpenAI

In [11]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API Key Exist")
else:
    print("Loaded API Key")

openai = OpenAI()

Loaded API Key


In [12]:
message = 'Hello GPT !!'
response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages=[{"role":"user","content":message}]
)
print(response.choices[0].message.content)

Hello! How can I assist you today?


## 1st Project

In [33]:
# Class to represent a webpage
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self,url):
        """Creating a website object by giving URL using Beautifulsoup library"""
        self.url = url
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.content,'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n",strip=True)


In [34]:
# Change the website and print Statement
forbes = Website("https://techcrunch.com")
print(forbes.title)
print(forbes.text)

TechCrunch | Startup and Technology News
TechCrunch Desktop Logo
TechCrunch Mobile Logo
Latest
Startups
Venture
Apple
Security
AI
Apps
Events
Podcasts
Newsletters
Sign In
Search
Submit
Site Search Toggle
Mega Menu Toggle
Topics
Latest
AI
Amazon
Apps
Biotech & Health
Climate
Cloud Computing
Commerce
Crypto
Enterprise
EVs
Fintech
Fundraising
Gadgets
Gaming
Google
Government & Policy
Hardware
Instagram
Layoffs
Media & Entertainment
Meta
Microsoft
Privacy
Robotics
Security
Social
Space
Startups
TikTok
Transportation
Venture
More from TechCrunch
Events
Startup Battlefield
StrictlyVC
Newsletters
Podcasts
Videos
Partner Content
TechCrunch Brand Studio
Crunchboard
Contact Us
Sign In
AI
OpenAI’s new GPT-4.1 AI models focus on coding
Kyle Wiggers
12 hours ago
Security
Silicon Valley crosswalk buttons hacked to imitate Musk, Zuckerberg’s voices
Zack Whittaker
15 hours ago
Transportation
Conifer locks down $20M seed round for its ‘drop-in’ electric hub motor
Sean O'Kane
16 hours ago
Top Headlines


Types of Prompts :

   System Prompts :
   Used to define the assistant's behavior, tone, expertise, or role before any user input.
   It sets the persona and boundaries.
   
   User Prompts :
   Represents the actual question or instruction from the user during interaction.

In [35]:
# Define a system prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [36]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [37]:
print(user_prompt_for(forbes))

You are looking at a website titled TechCrunch | Startup and Technology News
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

TechCrunch Desktop Logo
TechCrunch Mobile Logo
Latest
Startups
Venture
Apple
Security
AI
Apps
Events
Podcasts
Newsletters
Sign In
Search
Submit
Site Search Toggle
Mega Menu Toggle
Topics
Latest
AI
Amazon
Apps
Biotech & Health
Climate
Cloud Computing
Commerce
Crypto
Enterprise
EVs
Fintech
Fundraising
Gadgets
Gaming
Google
Government & Policy
Hardware
Instagram
Layoffs
Media & Entertainment
Meta
Microsoft
Privacy
Robotics
Security
Social
Space
Startups
TikTok
Transportation
Venture
More from TechCrunch
Events
Startup Battlefield
StrictlyVC
Newsletters
Podcasts
Videos
Partner Content
TechCrunch Brand Studio
Crunchboard
Contact Us
Sign In
AI
OpenAI’s new GPT-4.1 AI models focus on coding
Kyle Wiggers
12 hours ago
Security
Silicon Valley crosswalk b

 Messages : 
 OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [38]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 5 + 5?"}
]

In [39]:
# Caling OpenAI with system and user messages:
response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages)
print(response.choices[0].message.content)

Oh, I don’t know, let me consult my crystal ball… or just use basic math! It’s 10. Shocking, right?


In [40]:
def messages_for(website):
    return [
        {"role":"system","content":system_prompt},
        {"role": "user","content":user_prompt_for(website)}
    ]

In [41]:
messages_for(forbes)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled TechCrunch | Startup and Technology News\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nTechCrunch Desktop Logo\nTechCrunch Mobile Logo\nLatest\nStartups\nVenture\nApple\nSecurity\nAI\nApps\nEvents\nPodcasts\nNewsletters\nSign In\nSearch\nSubmit\nSite Search Toggle\nMega Menu Toggle\nTopics\nLatest\nAI\nAmazon\nApps\nBiotech & Health\nClimate\nCloud Computing\nCommerce\nCrypto\nEnterprise\nEVs\nFintech\nFundraising\nGadgets\nGaming\nGoogle\nGovernment & Policy\nHardware\nInstagram\nLayoffs\nMedia & Entertainment\nMeta\nMicrosoft\nPrivacy\nRobotics\nSecurity\nSocial\nSpace\nStartups\nTikTok\nTransportati

Lets summerize website passing with openAI API call

In [46]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [47]:
summarize("https://www.forbes.com/")

"# Forbes Website Summary\n\nForbes is a comprehensive news and information platform that focuses on business, finance, technology, and lifestyle through various sections, including:\n\n- **Billionaires**: Insights into the world's wealthiest individuals, featuring lists like the Forbes 400 and various country-specific rankings.\n- **Innovation**: Articles covering advancements in technology, AI, big data, and more.\n- **Leadership**: Topics on executive management strategies, diversity, and career development.\n- **Money**: Personal finance tips, investment strategies, and market analysis.\n- **Business**: Coverage of various industries, including aerospace, retail, and healthcare.\n- **Lifestyle**: Features on arts, travel, fashion, and wellness.\n\n## Recent News Highlights\n- **Trump Administration** has frozen $2.2 billion in federal funding to Harvard.\n- Sean ‘Diddy’ Combs pleads not guilty to new federal charges.\n- Discussion around tariffs under the Trump administration has s

In [48]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [49]:
display_summary("https://www.forbes.com/")

# Forbes Website Summary

Forbes is a global media company that provides news and analysis on business, finance, technology, and lifestyle. It features a wide variety of content categories, including:

## Key Categories
- **Billionaires**: Lists of the world's wealthiest individuals across various regions.
- **Innovation**: Covers technological advancements including AI, big data, and digital transformation.
- **Leadership**: Insights on leadership strategies, diversity, equity, and inclusion.
- **Money**: Personal finance, investment trends, retirement planning, and fintech.
- **Business**: Sector-specific news including aerospace, energy, media, and small businesses.
- **Lifestyle**: Topics on arts, travel, dining, style, and wellness.
- **Real Estate**: Residential and commercial property news.
- **Crypto & Digital Assets**: Information on cryptocurrency, NFTs, and blockchain.

## Recent News Highlights
- **Trump Administration**: A freeze on $2.2 billion in federal funding to Harvard after non-compliance with administrative demands.
- **Poll Insights**: Current polls indicate American concerns regarding tariffs and potential price increases.
- **Nvidia's Developments**: Announcement of AI supercomputers being manufactured entirely in the U.S.
- **Stock Market Updates**: Discussion on the largest gains in stock market history under Trump, while stocks remain volatile.
- **Cultural Events**: Coverage of notable individuals, including Sean "Diddy" Combs's legal challenges and high-profile events such as Katy Perry's space mission.

## Offered Services
- **Newsletters**: Various subscription options for tailored news updates.
- **Forbes Marketplace**: A hub for products and services.
- **Forbes Vetted**: A product review segment for consumer goods.

Forbes continues to be a leading source for business insights, lifestyle trends, and financial advice, catering to a global audience interested in wealth and innovation.